# Predict Winner from Realtime Stats using RNN

In [ ]:
import torch
from torch import nn
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm


BATCH_SIZE=32
PREDICT_TIME=15

## Dataset

In [ ]:
class LOLDataset(Dataset):
    def __init__(self,moneyData,label):
        self.money=[]
        for i in range(PREDICT_TIME):
            self.money.append([line[i] for line in moneyData])
        self.label=[i for i in label]
    
    def __getitem__(self,item):
        # print(item,"###")
        return torch.tensor([[torch.scalar_tensor(x[item])] for x in self.money]),torch.tensor(self.label[item])
    
    def __len__(self):
        return len(self.label)
            

In [ ]:
df = pd.read_csv('../data/LeagueofLegends.csv', sep=',')
from ast import literal_eval
df['golddiff'] = df['golddiff'].apply(literal_eval)
df['golddiff'].head()
# df["bResult"]

0    [0, 0, -14, -65, -268, -431, -488, -789, -494,...
1    [0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...
2    [0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...
3    [0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...
4    [40, 40, 44, -36, 113, 158, -121, -191, 23, 20...
Name: golddiff, dtype: object

Normalize data:

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for row in df['golddiff']:
    scaler.partial_fit(np.asanyarray(row).reshape(-1, 1))

golddiff = [scaler.transform(np.asanyarray(row).reshape(-1, 1)).reshape(-1) for row in df['golddiff']]
print(len(golddiff))

7620


Split the dataset into Train:Held-Out:Test = 6:2:2.

In [ ]:
dataset = LOLDataset(golddiff, df["bResult"])
test_size = valid_size = int(0.2 * len(dataset))
train_size = len(dataset)- test_size - valid_size

trainDataset, validDataset, testDataset = random_split(
    dataset = dataset,
    lengths = [train_size, valid_size, test_size],
    generator = torch.Generator().manual_seed(0)
)

trainLoader = DataLoader(trainDataset, batch_size = BATCH_SIZE, shuffle=True)
validLoader = DataLoader(validDataset, batch_size = BATCH_SIZE)
testLoader = DataLoader(testDataset, batch_size = BATCH_SIZE)

## RNN Network

In [ ]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()

        self.rnn=nn.LSTM(
            input_size=1,
            hidden_size=128,
            num_layers=1,
            batch_first=True
        )

        self.out = nn.Linear(128, 2)
    
    def forward(self,x):
        r_out,(h_n, h_c)=self.rnn(x,None)
        out = self.out(r_out[:, -1, :])
        return out

In [ ]:
model=RNN()
print(model)

RNN(
  (rnn): LSTM(1, 128, batch_first=True)
  (out): Linear(in_features=128, out_features=2, bias=True)
)


## Train

In [ ]:
def train(dataloader, model, loss_fn, optimizer, mute = False):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        x, y = Variable(x), Variable(y)

        predict = model(x)
        loss = loss_fn(predict, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 50 == 0 and not mute:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn, validation = False):
    model.eval()
    size = len(dataloader.dataset)

    correct = 0
    test_loss = 0
    with torch.no_grad():
        for step,(x,y) in enumerate(dataloader):
            x, y = Variable(x), Variable(y)
            predict = model(x)
            # print(predict)
            test_loss += loss_fn(predict, y).item()
            correct += (predict.argmax(1) == y).sum().item()
    
    print(f"{'Valid' if validation else 'Test'} Acc:{correct/size:>7f}, Avg Loss: {test_loss/size:>7f}")

In [387]:
MUTE = False
EPOCH = 5
LR = 0.00

torch.manual_seed(0)

optimizer = torch.optim.Adam(model.parameters(), lr = LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(1, EPOCH + 1):
    print(f"---------- Epoch #{epoch} ----------")
    train(trainLoader, model, loss_func, optimizer, mute = MUTE)
    test(validLoader, model, loss_func, validation = True)


---------- Epoch #1 ----------
loss: 0.682092  [    0/ 4572]
loss: 0.686725  [ 1600/ 4572]
loss: 0.692290  [ 3200/ 4572]
Valid Acc:0.549869, Avg Loss: 0.021722
---------- Epoch #2 ----------
loss: 0.691625  [    0/ 4572]
loss: 0.688258  [ 1600/ 4572]
loss: 0.692229  [ 3200/ 4572]
Valid Acc:0.549869, Avg Loss: 0.021722
---------- Epoch #3 ----------
loss: 0.690403  [    0/ 4572]
loss: 0.690360  [ 1600/ 4572]
loss: 0.691692  [ 3200/ 4572]
Valid Acc:0.549869, Avg Loss: 0.021722
---------- Epoch #4 ----------
loss: 0.689237  [    0/ 4572]
loss: 0.700754  [ 1600/ 4572]


KeyboardInterrupt: 

## Test

In [ ]:
test(testLoader, model, loss_func)

Test Acc:0.469816, Avg Loss: 0.022122
